### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2025, 1, 31), datetime.date(2025, 1, 30))

In [3]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-01-31
yesterday: 2025-01-30


In [4]:
# If run after actual work day
#today = today - timedelta(days=2)
today

datetime.date(2025, 1, 31)

### Restart and Run All Cells

In [6]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [9]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(20, 8)

In [10]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [11]:
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [12]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(20, 17)

In [13]:
df_merge['spd'] = 0

In [14]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(20, 12)

In [15]:
file_name = "Price-for-Order.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [16]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\Price-for-Order.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-for-Order.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-for-Order.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-for-Order.csv


In [17]:
df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(god_file, header=True, index=False)
df_merge[colo].to_csv(icd_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [19]:
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [20]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [21]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,3BBIF,0,RD50Pct,SET,10000,5.25,5.25,0.0,0.0,1,2024-11-13
B,AIMIRT,-3,RND05pct,SET,5000,10.2,10.5,0.1,0.961538,1,2024-11-26
B,CPNREIT,-13,RD40Pct,SET,5000,10.8,12.1,-0.4,-3.2,0,2024-11-26
B,DIF,-2,RD40Pct,SET,5000,8.3,8.4,0.0,0.0,1,2024-11-14
B,GVREIT,-10,RND20Pct,SET,6000,6.25,6.75,0.1,1.503759,1,2024-08-28
B,JMART,3,RND65pct,SET50,3200,11.2,10.9,-0.4,-3.539823,1,2024-09-30
B,KCE,8,RD70Pct,SET100,1000,22.8,22.0,-0.2,-0.900901,1,2024-08-26
B,MCS,-8,RD50Pct,SET,5000,5.8,6.2,-0.05,-0.8,1,2024-08-23
B,NER,-10,RD40pct,SET,6000,4.7,4.9,-0.06,-1.209677,0,2024-08-23
B,SENA,-17,RD60pct,SET,7000,1.92,2.18,0.0,0.0,0,2024-05-10
B,TFFIF,-6,RD20Pct,SET,5000,6.0,6.3,0.0,0.0,1,2024-12-12
B,WHAIR,-9,RD30pct,SET,5000,6.0,6.45,0.0,0.0,1,2024-11-21
S,AWC,90,COS,SET50,9000,4.96,3.16,-0.02,-0.628931,0,2024-05-09
S,BCH,74,COS,SET100,2000,21.7,14.3,0.0,0.0,0,2024-08-29
S,JMT,203,COS,SET50,900,50.0,14.7,-0.6,-3.921569,0,2024-08-22
S,

In [22]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [23]:
file_name   = 'orders-log.csv'
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\orders-log.csv


In [24]:
df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,3BBIF,0,RD50Pct,SET,10000,5.25,5.25,0.00,0.00,1,2024-11-13,52500.00
1,B,AIMIRT,-3,RND05pct,SET,5000,10.20,10.50,0.10,0.96,1,2024-11-26,51000.00
3,B,DIF,-2,RD40Pct,SET,5000,8.30,8.40,0.00,0.00,1,2024-11-14,41500.00
5,B,JMART,3,RND65pct,SET50,3200,11.20,10.90,-0.40,-3.54,1,2024-09-30,35840.00


In [25]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [26]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,3BBIF,0,RD50Pct,SET,10000,5.25,5.25,0.00,0.00,1,2024-11-13,52500.00
1,B,AIMIRT,-3,RND05pct,SET,5000,10.20,10.50,0.10,0.96,1,2024-11-26,51000.00
3,B,DIF,-2,RD40Pct,SET,5000,8.30,8.40,0.00,0.00,1,2024-11-14,41500.00
4,B,GVREIT,-10,RND20Pct,SET,6000,6.25,6.75,0.10,1.50,1,2024-08-28,37500.00
5,B,JMART,3,RND65pct,SET50,3200,11.20,10.90,-0.40,-3.54,1,2024-09-30,35840.00
6,B,KCE,8,RD70Pct,SET100,1000,22.80,22.00,-0.20,-0.90,1,2024-08-26,22800.00
7,B,MCS,-8,RD50Pct,SET,5000,5.80,6.20,-0.05,-0.80,1,2024-08-23,29000.00
10,B,TFFIF,-6,RD20Pct,SET,5000,6.00,6.30,0.00,0.00,1,2024-12-12,30000.00
11,B,WHAIR,-9,RD30pct,SET,5000,6.00,6.45,0.00,0.00,1,2024-11-21,30000.00


In [27]:
df.query('name == "AIMIRT"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
1,B,AIMIRT,-3,RND05pct,SET,5000,10.20,10.50,0.10,0.96,1,2024-11-26,51000.00


In [28]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
2,B,CPNREIT,-13,RD40Pct,SET,5000,10.80,12.10,-0.40,-3.20,0,2024-11-26,54000.00
8,B,NER,-10,RD40pct,SET,6000,4.70,4.90,-0.06,-1.21,0,2024-08-23,28200.00
9,B,SENA,-17,RD60pct,SET,7000,1.92,2.18,0.00,0.00,0,2024-05-10,13440.00
12,S,AWC,90,COS,SET50,9000,4.96,3.16,-0.02,-0.63,0,2024-05-09,44640.00
13,S,BCH,74,COS,SET100,2000,21.70,14.30,0.00,0.00,0,2024-08-29,43400.00
14,S,JMT,203,COS,SET50,900,50.00,14.70,-0.60,-3.92,0,2024-08-22,45000.00
15,S,PTG,64,COS,SET100,3600,11.40,7.45,-0.25,-3.25,0,2024-11-26,41040.00
16,S,RCL,65,RD60pct,SET100,3000,38.75,24.00,-0.30,-1.23,0,2024-11-21,116250.00
17,S,SYNEX,-484,RD70pct,SET,2000,0.00,13.40,0.20,1.52,0,2024-08-27,0.00
18,S,TVO,41,DTD,SET,4000,25.25,21.00,-0.20,-0.94,0,2025-03-13,101000.00


In [29]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [30]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
5,B,JMART,3,RND65pct,SET50,3200,11.20,10.90,-0.40,-3.54,1,2024-09-30,35840.00
6,B,KCE,8,RD70Pct,SET100,1000,22.80,22.00,-0.20,-0.90,1,2024-08-26,22800.00


In [31]:
df.query('trade == "S" & spd < 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
17,S,SYNEX,-484,RD70pct,SET,2000,0.00,13.40,0.20,1.52,0,2024-08-27,0.00


In [32]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

In [33]:
file_name = "orders-log.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [34]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\orders-log.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders-log.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders-log.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders-log.csv


In [35]:
df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(god_file, header=True, index=False)
df[colx].to_csv(icd_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [37]:
df['percent'] = df['%change']

In [38]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
2,B,CPNREIT,-13,RD40Pct,SET,5000,10.80,12.10,-0.40,-3.20,0,2024-11-26,54000.00,-3.20
5,B,JMART,3,RND65pct,SET50,3200,11.20,10.90,-0.40,-3.54,1,2024-09-30,35840.00,-3.54
14,S,JMT,203,COS,SET50,900,50.00,14.70,-0.60,-3.92,0,2024-08-22,45000.00,-3.92
15,S,PTG,64,COS,SET100,3600,11.40,7.45,-0.25,-3.25,0,2024-11-26,41040.00,-3.25


In [39]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent


In [40]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
6,B,KCE,8,RD70Pct,SET100,1000,22.80,22.00,-0.20,-0.90,1,2024-08-26,22800.00,-0.90
5,B,JMART,3,RND65pct,SET50,3200,11.20,10.90,-0.40,-3.54,1,2024-09-30,35840.00,-3.54
0,B,3BBIF,0,RD50Pct,SET,10000,5.25,5.25,0.00,0.00,1,2024-11-13,52500.00,0.00
3,B,DIF,-2,RD40Pct,SET,5000,8.30,8.40,0.00,0.00,1,2024-11-14,41500.00,0.00
1,B,AIMIRT,-3,RND05pct,SET,5000,10.20,10.50,0.10,0.96,1,2024-11-26,51000.00,0.96
10,B,TFFIF,-6,RD20Pct,SET,5000,6.00,6.30,0.00,0.00,1,2024-12-12,30000.00,0.00
7,B,MCS,-8,RD50Pct,SET,5000,5.80,6.20,-0.05,-0.80,1,2024-08-23,29000.00,-0.80
11,B,WHAIR,-9,RD30pct,SET,5000,6.00,6.45,0.00,0.00,1,2024-11-21,30000.00,0.00
4,B,GVREIT,-10,RND20Pct,SET,6000,6.25,6.75,0.10,1.50,1,2024-08-28,37500.00,1.50
8,B,NER,-10,RD40pct,SET,6000,4.70,4.90,-0.06,-1.21,0,2024-08-23,28200.00,-1.21


### Sell recently purchased stocks if profitable

In [42]:
#pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(114, 4)

In [43]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [44]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(110, 8)

In [45]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending=False)

,name,date_x,qty,cost,price,difa,%change,%profit
31,GVREIT,2023-09-07,3000,6.60,6.75,450.00,1.50,2.27
32,GVREIT,2023-09-11,3000,6.60,6.75,450.00,1.50,2.27
33,GVREIT,2023-09-22,6000,6.10,6.75,3900.00,1.50,10.66
8,AIMIRT,2024-06-17,2500,10.50,10.50,0.00,0.96,0.00


### Profit per Month

In [47]:
year = 2025
month = 1
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 1
ORDER BY sells.date DESC, name


In [48]:
sells_df

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days


In [49]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: 0.00


In [50]:
file_name = "monthly-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [51]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\monthly-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\monthly-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\monthly-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\monthly-sales.csv


In [52]:
sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(god_file, header=True, index=False)
sells_df.to_csv(icd_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [53]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day


In [54]:
sells_df.gross.sum()

0

In [55]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: nan%


In [56]:
#dt = date(2023, 11, 8)
dt = today
dt.strftime("%B %d, %Y")

'January 31, 2025'

In [57]:
print("Hello, {}! Today is {:%B %d, %Y}.".format("Alice", dt))

Hello, Alice! Today is January 31, 2025.
